<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft

In [0]:
%cd ..
!ls


/content
conceptor  sample_data


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
#load gensim formatted Full Glove embeddings
!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2
#import gensim

from gensim.models.keyedvectors import KeyedVectors

resourceFile = '/content/'


#glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

Downloading...
From: https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2
To: /content/gensim_glove.840B.300d.txt.bin
2.65GB [00:17, 151MB/s]
The glove embedding has been loaded!


In [0]:
#load gensim formatted Full Word2vec embeddings
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
!gunzip GoogleNews-vectors-negative300.bin.gz
  
import gensim

from gensim.models.keyedvectors import KeyedVectors

resourceFile = '/content/'


#word2vec = KeyedVectors.load_word2vec_format(resourceFile + 'GoogleNews-vectors-negative300.bin', binary=True)
print('The glove embedding has been loaded!')

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:10, 157MB/s]
The glove embedding has been loaded!


In [0]:

!gdown https://drive.google.com/uc?id=1U_UGB2vyTuTIcbV_oeDtJCtAtlFMvXOM
!gdown https://drive.google.com/uc?id=1j_b4TRpL3f0HQ8mV17_CtOXp862YjxxB

Downloading...
From: https://drive.google.com/uc?id=1U_UGB2vyTuTIcbV_oeDtJCtAtlFMvXOM
To: /content/small_glove.txt
333MB [00:02, 160MB/s]
Downloading...
From: https://drive.google.com/uc?id=1j_b4TRpL3f0HQ8mV17_CtOXp862YjxxB
To: /content/small_word2vec.txt
267MB [00:01, 164MB/s]


In [0]:
def load_embd(model_str):
  embeddings = {}
  f = codecs.open("/content/small_" + model_str + ".txt","r","utf-8")
  
  for line in f:
    line = line.split(" ",1)
    embeddings[line[0].lower()] = np.fromstring(line[1],sep=" ",dtype="float32")
  return embeddings

In [0]:
glove_embd = load_embd("glove")
#w2v_embd = load_embd("word2vec")

2.8722813232690143


In [0]:
x = np.array(list(glove_embd.values())).T
for ind in np.arange(0,x.shape[1]):
  curr_word = list(glove_embd.keys())[ind]
  print(curr_word)

In [0]:
print("Hello")
def post_process_cn(orig_embd, saveto, alpha = 2):
  print("starting...")
  #x = orig_embd.vectors
  print(orig_embd.vectors.shape)
  #Calculate the correlation matrix
  R = orig_embd.vectors.dot(orig_embd.vectors.T)/(orig_embd.vectors.shape[1])
  
  #Calculate the conceptor matrix
  C = R @ (np.linalg.inv(R + alpha ** (-2) * np.eye(orig_embd.vectors.shape[0])))
  
  #Calculate the negation of the conceptor matrix
  negC = np.eye(orig_embd.vectors.shape[0]) - C
  
  #Post-process the vocab matrix
  newX = (negC @ orig_embd.vectors).T
  print(newX.shape)
  new_embd = {}
  count = 0
  #Convert to dictonary
  for ind in np.arange(0,orig_embd.vectors.shape[1]):
    #curr_word = list(orig_embd.keys())[ind]
    #new_embd[curr_word] = newX[ind,:]
    #count = count + 1
    
    #Directly store post-processed embeddings to file
    f = open("/content/" + saveto, "a+")
    val = ' '.join((np.array2string(newX[ind,:]).replace("[","").replace("]","")).split()) + "\n"
    f.write(curr_word + " " + val)
    count = count + 1
    print(count)
    
  
  return new_embd

print("Begin")
glove_cn = post_process_cn(glove_embd, "full_glove_cn.txt")
print("Glove done")
#w2v_cn = post_process_cn(word2vec, "full_word2vec_cn.txt")
#print("Word2vec done")
  

In [0]:
def post_process_cn_matrix(x, alpha = 2):
  print("starting...")
  #x = orig_embd.vectors
  print(x.shape)
  
  #Calculate the correlation matrix
  R = x.dot(x.T)/(x.shape[1])
  print("R calculated")
  print('Memory', psutil.virtual_memory())
  #Calculate the conceptor matrix
  C = R @ (np.linalg.inv(R + alpha ** (-2) * np.eye(x.shape[0])))
  print("C calculated")
  print('Memory', psutil.virtual_memory())
  #Calculate the negation of the conceptor matrix
  negC = np.eye(x.shape[0]) - C
  print("negC calculated")
  print('Memory', psutil.virtual_memory())
  #Post-process the vocab matrix
  newX = (negC @ x).T
  print(newX.shape)
  return newX

Save post-processed embeddings to text file

In [0]:
def convert_to_text(curr_embd, saveto):
  for key, value in curr_embd.items():
    #print("key: ", key, " value: ", np.array_str(value).replace("[","").replace("]",""))
    f = open("/content/" + saveto, "a+")
    val = ' '.join((np.array2string(value).replace("[","").replace("]","")).split()) + "\n"
    f.write(key + " " + val)
    #with open("/content/" + saveto) as f:
      #f.write(key + " " + np.array_str(value).replace("[","").replace("]",""))

convert_to_text(glove_cn, "glove_cn.txt")
convert_to_text(w2v_cn, "w2v_cn.txt")

# **Intrinsic Evaluation**
Evaluate embeddings for word similarity 

In [0]:
def similarity_eval(dataSetAddress, embd):
    wordVecModel = eval(embd)
    vocab = set(list(wordVecModel.keys()))
    
    fread_simlex = open(dataSetAddress, "r")
    
    pair_list = []

    line_number = 0
    for line in fread_simlex:
      #print(line)
      if line_number > 0:
          tokens = line.split()
          word_i = tokens[0]
          word_j = tokens[1]
          score = float(tokens[2])
          if word_i in vocab and word_j in vocab:
              pair_list.append( ((word_i, word_j), score) )
      line_number += 1

    pair_list.sort(key=lambda x: - x[1]) # order the pairs from highest score (most similar) to lowest score (least similar)


    extracted_scores = {}

    extracted_list = []
    
               
    for (x,y) in pair_list:
        (word_i, word_j) = x
        
        current_distance = 1- cosine_similarity( wordVecModel[word_i].reshape(1,-1)  , wordVecModel[word_j].reshape(1,-1) )        

        extracted_scores[(word_i, word_j)] = current_distance
        extracted_list.append(((word_i, word_j), current_distance))

    extracted_list.sort(key=lambda x: x[1])

    spearman_original_list = []
    spearman_target_list = []

    for position_1, (word_pair, score_1) in enumerate(pair_list):
        score_2 = extracted_scores[word_pair]
        position_2 = extracted_list.index((word_pair, score_2))
        spearman_original_list.append(position_1)
        spearman_target_list.append(position_2)

    spearman_rho = spearmanr(spearman_original_list, spearman_target_list)
    
    return spearman_rho[0]

In [0]:
!wget https://raw.githubusercontent.com/mfaruqui/eval-word-vectors/master/data/word-sim/EN-RG-65.txt https://raw.githubusercontent.com/mfaruqui/eval-word-vectors/master/data/word-sim/EN-WS-353-ALL.txt https://raw.githubusercontent.com/mfaruqui/eval-word-vectors/master/data/word-sim/EN-RW-STANFORD.txt https://raw.githubusercontent.com/mfaruqui/eval-word-vectors/master/data/word-sim/EN-MEN-TR-3k.txt https://raw.githubusercontent.com/mfaruqui/eval-word-vectors/master/data/word-sim/EN-MTurk-287.txt https://raw.githubusercontent.com/mfaruqui/eval-word-vectors/master/data/word-sim/EN-SIMLEX-999.txt https://raw.githubusercontent.com/mfaruqui/eval-word-vectors/master/data/word-sim/EN-SimVerb-3500.txt -P wordSimData1

In [0]:
!ls
!cd wordSimData/
!ls

EN-RG-65.txt  sample_data  small_glove.txt  small_word2vec.txt	wordSimData
EN-RG-65.txt  sample_data  small_glove.txt  small_word2vec.txt	wordSimData


In [0]:
dataSets = ['EN-WS-353-ALL.txt', 'EN-RW-STANFORD.txt', 'EN-MEN-TR-3k.txt', 'EN-MTurk-287.txt', 'EN-SIMLEX-999.txt', 'EN-SimVerb-3500.txt', 'EN-RG-65.txt']

for dataset in dataSets:
    dataSetAddress = 'wordSimData1/' +  dataset
    print('evaluating the data set', dataset)
    
    print('Word2Vec + CN : %.4f' %  similarity_eval(dataSetAddress, 'w2v_cn'))
    print('Glove + CN : %.4f' %  similarity_eval(dataSetAddress, 'glove_cn'))
        
    print('\n')

evaluating the data set EN-WS-353-ALL.txt
Word2Vec + CN : 0.6930
Glove + CN : 0.7908


evaluating the data set EN-RW-STANFORD.txt
Word2Vec + CN : 0.5804
Glove + CN : 0.5898


evaluating the data set EN-MEN-TR-3k.txt
Word2Vec + CN : 0.7867
Glove + CN : 0.8338


evaluating the data set EN-MTurk-287.txt
Word2Vec + CN : 0.6681
Glove + CN : 0.7107


evaluating the data set EN-SIMLEX-999.txt
Word2Vec + CN : 0.4682
Glove + CN : 0.4853


evaluating the data set EN-SimVerb-3500.txt
Word2Vec + CN : 0.3830
Glove + CN : 0.3636


evaluating the data set EN-RG-65.txt
Word2Vec + CN : 0.7892
Glove + CN : 0.7840




# Extrinsic Evaluation


To download post-processed embeddings, run the following cell

In [0]:
!gdown https://drive.google.com/uc?id=1p7ThmH_Qzk8AsfHU-8Mh15nUU6MpSclE
!gdown https://drive.google.com/uc?id=16fRNeAD0kmsXB9GVEZ-i0xH9IkRn9TTV

Downloading...
From: https://drive.google.com/uc?id=1p7ThmH_Qzk8AsfHU-8Mh15nUU6MpSclE
To: /content/glove_cn.txt
524MB [00:04, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=16fRNeAD0kmsXB9GVEZ-i0xH9IkRn9TTV
To: /content/w2v_cn.txt
310MB [00:01, 156MB/s]


Clone git repository for extrinsic evaluation

In [0]:
!git clone https://github.com/karvesaket/conceptor.git
  

Cloning into 'conceptor'...
remote: Enumerating objects: 279, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (249/249), done.
remote: Total 279 (delta 120), reused 35 (delta 5), pack-reused 0
Receiving objects: 100% (279/279), 3.96 MiB | 3.52 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [0]:
%cd ..
%cd ..
%cd ..
!rm -r conceptor

/content/conceptor/Extrinsic-Evaluation-tasks-master
/content/conceptor
/content


In [0]:
%cd content

[Errno 2] No such file or directory: 'content'
/content


In [0]:
!ls conceptor/Extrinsic-Evaluation-tasks-master

README.md			  sentence_polarity_classification.zip
Relation_extraction		  sentiment_classification
Relation_extraction.zip		  snli
requirements.txt		  subjectivity_classification
sentence_polarity_classification


Upload modified files for all tasks

## Relation Extraction

In [0]:
%cd content
!unzip Relation_extraction.zip

[Errno 2] No such file or directory: 'content'
/content
Archive:  Relation_extraction.zip
   creating: Relation_extraction/
   creating: Relation_extraction/embeddings/
  inflating: Relation_extraction/embeddings/.gitignore  
   creating: __MACOSX/
   creating: __MACOSX/Relation_extraction/
   creating: __MACOSX/Relation_extraction/embeddings/
  inflating: __MACOSX/Relation_extraction/embeddings/._.gitignore  
  inflating: __MACOSX/Relation_extraction/._embeddings  
  inflating: Relation_extraction/requirements.txt  
  inflating: __MACOSX/Relation_extraction/._requirements.txt  
  inflating: Relation_extraction/preprocess.py  
  inflating: __MACOSX/Relation_extraction/._preprocess.py  
   creating: Relation_extraction/dataset/
  inflating: Relation_extraction/dataset/train.txt  
   creating: __MACOSX/Relation_extraction/dataset/
  inflating: __MACOSX/Relation_extraction/dataset/._train.txt  
  inflating: Relation_extraction/dataset/test.txt  
  inflating: __MACOSX/Relation_extraction/d

In [0]:
%cd conceptor/Extrinsic-Evaluation-tasks-master/Relation_extraction

/content/conceptor/Extrinsic-Evaluation-tasks-master/Relation_extraction


In [0]:
aaa = [0.7650, 0.7490, 0.7409, 0.7512, 0.7460, 0.7505, 0.7446, 0.7541, 0.7516, 0.7519]
bbb = [0.7266, 0.7250, 0.7072, 0.7144, 0.7138, 0.7195, 0.7235, 0.7222, 0.7221, 0.7198]
print(np.mean(aaa))
print(np.std(aaa))
print(np.mean(bbb))
print(np.std(bbb))

0.7504799999999999
0.006134623052804467
0.7194100000000001
0.005661351428766797


In [0]:
!python3 preprocess.py

{'PADDING': 0, 'LowerMin': 1, 'GreaterMax': 2, -30: 3, -29: 4, -28: 5, -27: 6, -26: 7, -25: 8, -24: 9, -23: 10, -22: 11, -21: 12, -20: 13, -19: 14, -18: 15, -17: 16, -16: 17, -15: 18, -14: 19, -13: 20, -12: 21, -11: 22, -10: 23, -9: 24, -8: 25, -7: 26, -6: 27, -5: 28, -4: 29, -3: 30, -2: 31, -1: 32, 0: 33, 1: 34, 2: 35, 3: 36, 4: 37, 5: 38, 6: 39, 7: 40, 8: 41, 9: 42, 10: 43, 11: 44, 12: 45, 13: 46, 14: 47, 15: 48, 16: 49, 17: 50, 18: 51, 19: 52, 20: 53, 21: 54, 22: 55, 23: 56, 24: 57, 25: 58, 26: 59, 27: 60, 28: 61, 29: 62, 30: 63}


In [0]:
!python3 train_cnn.py

{'PADDING': 0, 'LowerMin': 1, 'GreaterMax': 2, -30: 3, -29: 4, -28: 5, -27: 6, -26: 7, -25: 8, -24: 9, -23: 10, -22: 11, -21: 12, -20: 13, -19: 14, -18: 15, -17: 16, -16: 17, -15: 18, -14: 19, -13: 20, -12: 21, -11: 22, -10: 23, -9: 24, -8: 25, -7: 26, -6: 27, -5: 28, -4: 29, -3: 30, -2: 31, -1: 32, 0: 33, 1: 34, 2: 35, 3: 36, 4: 37, 5: 38, 6: 39, 7: 40, 8: 41, 9: 42, 10: 43, 11: 44, 12: 45, 13: 46, 14: 47, 15: 48, 16: 49, 17: 50, 18: 51, 19: 52, 20: 53, 21: 54, 22: 55, 23: 56, 24: 57, 25: 58, 26: 59, 27: 60, 28: 61, 29: 62, 30: 63}
Using TensorFlow backend.
Enter embedding typeglove
tcmalloc: large alloc 2635227136 bytes == 0x49ac000 @  0x7faa38737001 0x7faa3646bb85 0x7faa364ceb43 0x7faa364d0a86 0x7faa36568868 0x5030d5 0x507641 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x506393 0x634d52 0x634e0a 0x6385c8 0x63915a 0x4a6f10 0x7faa38332b97 0x5afa0a
300
The embedding has been loaded from gensim!
Load dataset
Max Sentence Lengths:  [97, 67]
Conceptor?

## Sentence Polarity classification

In [0]:
a = [1,2,3,4]
print(np.multiply(a,10))

[10 20 30 40]


In [0]:
%cd conceptor/Extrinsic-Evaluation-tasks-master
%cd sentence_polarity_classification
!ls

/content/conceptor/Extrinsic-Evaluation-tasks-master
/content/conceptor/Extrinsic-Evaluation-tasks-master/sentence_polarity_classification
data  pkl  preprocess.py  train.py


In [0]:
!python3 preprocess.py
!python3 train.py

data/train.txt 5000 sentences
data/dev.txt 2500 sentences
data/test.txt 2500 sentences
Enter embedding typeword2vec
tcmalloc: large alloc 3600007168 bytes == 0x3842000 @  0x7f10af779001 0x7f10ad4adb85 0x7f10ad510b43 0x7f10ad512a86 0x7f10ad5aa868 0x5030d5 0x507641 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x506393 0x634d52 0x634e0a 0x6385c8 0x63915a 0x4a6f10 0x7f10af374b97 0x5afa0a
The embedding has been loaded from gensim!
Load dataset
Load pre-trained embeddings file
Conceptor? n
Embeddings shape:  (17469, 300)
Len words:  23908
Unknown tokens: 26.88%
Unknown tokens: 26.75%
Unknown tokens: 26.84%
Data stored in pkl folder
Using TensorFlow backend.
data loaded!
Longest sentence: 120
X_train shape: (5000, 120)
X_dev shape: (2500, 120)
X_test shape: (2500, 120)
y_train shape: (5000,)
Build model...
2019-02-01 00:35:39.846246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but

In [0]:
%cd ..

/content/conceptor/Extrinsic-Evaluation-tasks-master


## Sentiment Classification

In [0]:
!ls

 1F67jJJVCXjXUxPzQ2Q9ug1c5lMvu9Yqz   sample_data	 'view?usp=sharing'
 conceptor			     small_glove.txt	  w2v_cn.txt
 glove_cn.txt			     small_word2vec.txt


In [0]:
%cd conceptor/Extrinsic-Evaluation-tasks-master/sentiment_classification

/content/conceptor/Extrinsic-Evaluation-tasks-master/sentiment_classification


In [0]:
!python3 train.py

Using TensorFlow backend.
Memory svmem(total=13655322624, available=12675645440, percent=7.2, used=683270144, free=1344483328, active=900517888, inactive=10868092928, buffers=59322368, cached=11568246784, shared=905216, slab=421666816)
Enter embedding typeword2vec
Conceptor?y
tcmalloc: large alloc 3600007168 bytes == 0x5ea2000 @  0x7f3d8f0dc001 0x7f3d8ce10b85 0x7f3d8ce73b43 0x7f3d8ce75a86 0x7f3d8cf0d868 0x5030d5 0x507641 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x506393 0x634d52 0x634e0a 0x6385c8 0x63915a 0x4a6f10 0x7f3d8ecd7b97 0x5afa0a
The embedding has been loaded from gensim!
Memory svmem(total=13655322624, available=7991984128, percent=41.5, used=5536538624, free=152301568, active=6191370240, inactive=6901071872, buffers=59506688, cached=7906975744, shared=905216, slab=283123712)
300
1646592/1641221 [==============================] - 0s 0us/step
n_vocab 88586
Memory svmem(total=13655322624, available=12421451776, percent=9.0, used=4875046912

In [0]:
%cd ..

/content/conceptor/Extrinsic-Evaluation-tasks-master


## SNLI

In [0]:
%cd conceptor/Extrinsic-Evaluation-tasks-master/snli
#Download the SNLI data
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
!unzip snli_1.0.zip
!mv snli_1.0/* .

In [0]:
!python3 train.py

In [0]:
%cd ..

/content/conceptor/Extrinsic-Evaluation-tasks-master


## Subjectivity Classification

In [0]:
%cd conceptor/Extrinsic-Evaluation-tasks-master/subjectivity_classification

/content/conceptor/Extrinsic-Evaluation-tasks-master/subjectivity_classification


In [0]:
!python3 preprocess.py
!python3 cnn.py

Enter embeddings file name: j
data/train.txt 5330 sentences
data/dev.txt 2664 sentences
data/test.txt 2668 sentences
Enter embedding typeword2vec
tcmalloc: large alloc 3600007168 bytes == 0x4436000 @  0x7f7e3d5bb001 0x7f7e3b2efb85 0x7f7e3b352b43 0x7f7e3b354a86 0x7f7e3b3ec868 0x5030d5 0x507641 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x506393 0x634d52 0x634e0a 0x6385c8 0x63915a 0x4a6f10 0x7f7e3d1b6b97 0x5afa0a
The embedding has been loaded from gensim!
Load pre-trained embeddings file
Embeddings shape:  (15881, 300)
Len words:  21347
Conceptor? n
Unknown tokens: 26.72%
Unknown tokens: 26.46%
Unknown tokens: 26.43%
Data stored in pkl folder
Using TensorFlow backend.
data loaded!
Longest sentence: 59
X_train shape: (5330, 59)
X_dev shape: (2664, 59)
X_test shape: (2668, 59)
Build model...
2019-02-01 02:38:46.070770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there mus

In [0]:
%cd ..

# Results

## Small Dataset

1.   Relation extraction

>*   **Glove**
>> Accuracy = 0.7210 \\
>> F1 = 0.6978
*   **Glove + CN**
>> Accuracy = 0.7379 \\
>> F1 = 0.7010
*   **Word2vec**
>> Accuracy = 0.7530 \\
>> F1 = 0.7213
*   **Word2vec + CN**
>> Accuracy = 0.7483 \\
>> F1 = 0.7162

2.   Sentence Polarity Classification

>*   **Glove**
>> Dev-Accuracy = 0.9108 \\
>> Test-Accuracy = 0.8912
*   **Glove + CN**
>> Dev-Accuracy = 0.9060  \\
>> Test-Accuracy = 0.8936
*   **Word2vec**
>> Dev-Accuracy = 0.9004 \\
>> Test-Accuracy = 0.8980
*   **Word2vec + CN**
>> Dev-Accuracy = 0.9 \\
>> Test-Accuracy = 0.9068

3.   Sentiment Classification

>*   **Glove**
>> Test-Accuracy = 0.8096
*   **Glove + CN** 
>> Test-Accuracy = 0.8051
*   **Word2vec**
>> Test-Accurracy = 0.8111
*   **Word2vec + CN**
>> Test-Accurracy = 0.8130

4.   SNLI

>*   **Glove**
>> Test-Loss = 0.7576 \\
>> Test-Accuracy = 0.6868
*   **Glove + CN**
>> Test-Loss = 0.7643 \\
>> Test-Accuracy = 0.6822
*   **Word2vec**
>> Test-Loss = 0.7732 \\
>> Test-Accuracy = 0.6818
*   **Word2vec + CN**
>> Test-Loss = 0.7576 \\
>> Test-Accuracy = 0.6868

5.   Subjectivity Classification

>*   **Glove**
>> Dev-Accuracy = 0.7913 \\
>> Test-Accuracy = 0.7770
*   **Glove + CN**
>> Dev-Accuracy = 0.7932  \\
>> Test-Accuracy = 0.7837
*   **Word2vec**
>> Dev-Accuracy = 0.7834 \\
>> Test-Accuracy = 0.7811
*   **Word2vec + CN**
>> Dev-Accuracy = 0.7868 \\
>> Test-Accuracy = 0.7796


## Full Dataset

1.   Relation extraction

>*   **Glove**
>> Accuracy: Mean = 0.7505 | SD = 0.00613 \\
>> F1: Mean = 0.7194 | SD = 0.00566
*   **Glove + CN**
>> Accuracy: Mean = 0.7417 | SD = 0.00508 \\
>> F1: Mean = 0.7121 | SD = 0.00521
*   **Word2vec**
>> Accuracy: Mean = 0.7534 | SD = 0.00459 \\
>> F1: Mean = 0.7179 | SD = 0.00637
*   **Word2vec + CN**
>> Accuracy: Mean = 0.7450 | SD = 0.00495 \\
>> F1: Mean = 0.7095 | SD = 0.00713

2.   Sentence Polarity Classification

>*   **Glove**
>> Dev-Accuracy = Mean = 0.9108 | SD = 0.00259  \\
>> Test-Accuracy = Mean = 0.9023 | SD = 0.00230
*   **Glove + CN**
>> Dev-Accuracy = Mean = 0.9110 | SD = 0.00149 \\
>> Test-Accuracy = Mean = 0.9055 | SD = 0.00147
*   **Word2vec**
>> Dev-Accuracy = Mean = 0.9074 | SD = 0.00433 \\
>> Test-Accuracy = Mean = 0.9029 | SD = 0.00165
*   **Word2vec + CN**
>> Dev-Accuracy = Mean = 0.9053 | SD = 0.00350 \\
>> Test-Accuracy = Mean = 0.9026 | SD = 0.00294

3.   Sentiment Classification

>*   **Glove**
>> Test-Accuracy: Mean = 0.8068 | SD = 0.00562 \\
>> Test-Score: Mean = 0.8778 | SD = 0.1511
*   **Glove + CN** 
>> Test-Accuracy: Mean = 0.8159 | SD = 0.00512 \\
>> Test-Score: Mean = 0.8065 | SD = 0.1531
*   **Word2vec**
>> Test-Accuracy: Mean = 0.8216 | SD = 0.00366 \\
>> Test-Score: Mean = 0.7241 | SD = 0.1609
*   **Word2vec + CN**
>> Test-Accuracy: Mean = 0.8216 | SD = 0.00366 \\
>> Test-Score: Mean = 0.7241 | SD = 0.1609

4.   SNLI

>*   **Glove**
>> Test-Loss = 0.7576 \\
>> Test-Accuracy = 0.6868
*   **Glove + CN**
>> Test-Loss = 0.7643 \\
>> Test-Accuracy = 0.6822
*   **Word2vec**
>> Test-Loss = 0.7732 \\
>> Test-Accuracy = 0.6818
*   **Word2vec + CN**
>> Test-Loss = 0.7576 \\
>> Test-Accuracy = 0.6868

5.   Subjectivity Classification

>*   **Glove**
>> Dev-Accuracy = Mean = 0.7950 | SD = 0.00236 \\
>> Test-Accuracy = Mean = 0.7897 | SD = 0.00355
*   **Glove + CN**
>> Dev-Accuracy = Mean = 0.7991 | SD = 0.00252 \\
>> Test-Accuracy = Mean = 0.7903 | SD = 0.00411
*   **Word2vec**
>> Dev-Accuracy = Mean = 0.7894 | SD = 0.00234 \\
>> Test-Accuracy = Mean = 0.7843 | SD = 0.00311
*   **Word2vec + CN**
>> Dev-Accuracy = Mean = 0.7863 | SD = 0.00254 \\
>> Test-Accuracy = Mean = 0.7849 | SD = 0.00488





